In [74]:
#Define things, collect info to build tableau, validate input
while True:
    try:
        g = int(raw_input("The genus is:"))
        _a_ = [int(n) for n in raw_input("Enter a_1 through a_k-2 as a list of numbers separated by spaces:").split()]
        _b_ = [int(n) for n in raw_input("Enter b_1 through b_k-2 as a list of numbers separated by spaces:").split()]
        k = len(_a_)+2
    except ValueError: 
        print("Oops! That doesn't look right... Try again!")
        continue
        
    #Add validation to make sure each a_i, b_i is in the correct range, no duplicates, sort by order?
    
    #Remove boxes from each column (as a list)
    a = range(2,g+1) #Symbols in column 1 of Lambda
    for n in _a_:
        a.remove(n) #Symbols in column 1 of lambda_D
    b = range(1,g) #Symbols in column 0 of Lambda
    for n in _b_: 
        b.remove(n) #Symbols in column 0 of lambda_D  
        
    #Verify that the given values form a tableau    
    if all(a[n]>b[n] for n in range(g-k+1)) and len(a)==len(b):
        break
    else:
        print("Oops! This isn't a valid displacement tableau. Be sure the number of b-values smaller than each n is no more than the number of a values.")
        continue
        
#Construct torsion profile
def torsion(n):
    if a.count(n)==0 or b.count(n)==0: 
        return 0
    else: 
        return b.index(n)-a.index(n)+1 #(y in column 0)-(x in column 1) +1 horizontal unit gives lattice distance
    
_m_= [torsion(n) for n in range(2,g+1)]

def calc_torsion(n): #To use in places where we do modular calculations since mod 0 breaks things
    if a.count(n)==0 or b.count(n)==0:
        return 104729
    else: 
        return torsion(n)

print "_m_=", _m_
    
#Build lambda_D
lambda1 = Tableau([])
for n in range(g-k+1):
    lambda1 = lambda1.add_entry((n,0),b[n])
    lambda1 = lambda1.add_entry((n,1),a[n])

print "k=",k
print "D"
lambda1.pp()



#Calculate initial placement of each symbol
def xi1(s): #for s in range(1,g+1) 
    if s<g+1 and b.count(s)>0:
        return b.index(s)
    elif s<g+1 and a.count(s)>0:
        return a.index(s)-1
    else:
        return -1
    

#Calculate placement in subsequent tableaux
def xi(c,s):
    if torsion(s)>0:
         return ((c*xi1(s))-((c-1)*(s-1))) % _m_[s-2]
    else:
         return ((c*xi1(s))-((c-1)*(s-1)))

        
def tryj(i,j): #Build a tableau for the next value of c corresponding to a rank jump of j
    lambdac = Tableau([])
    coordinatesum = 0 #Tracks which diagonal we're on
    
    for coordinatesum in range(0,g): #range(0,(ranks[i-1]+heights[i-1])):
            x = max(0, (coordinatesum - (heights[i-1]-j-1))) #Returns 0 if coordinatesum is smaller than our max y value, 0 else
            y = coordinatesum - x
            
            while x < ranks[i-1]+k-j+1 and y>-1:
                #Find entries above and left of (x,y)
                if x == 0 and y == 0:
                    p = 0
                    q = 0
                elif x == 0:
                    p = 0
                    q = lambdac.entry((y-1,0))
                elif y == 0:
                    p = lambdac.entry((0,x-1))
                    q = 0
                else: 
                    if ((y-1,x) in lambdac.cells()) and ((y,x-1) in lambdac.cells()):
                        p = lambdac.entry((y,x-1))
                        q = lambdac.entry((y-1,x))
                        
                    else: #Break if we're missing a box, so the tableau can't be rectangular 
                        return lambdac
                        
                s = max(p,q)+1 #Smallest potential symbol to put in (x,y)

                while ((y-x) - xi(i,s)) % calc_torsion(s) != 0: #Check xi^c_s
                    if s>g:
                        
                        if lambdac.height() == 1: #this is probably a specific case bandaid for a general problem
                            return lambdac
                        break
                    else:
                        s+=1
                    
                if s < g+1:
                    lambdac=lambdac.add_entry((y,x), s) #Add smallest possible symbol that has correct xi^c, if it exists
                    if s == g:
                        return lambdac 
                    
                if x==ranks[i-1]+k-j and y==heights[i-1]-j-1: #if we've built a rectangular tableau corresponding to a rank jump of j
                    return lambdac
                    #break
                else:
                    x+=1
                    y-=1
            
            #coordinatesum += 1
    return lambdac

#Build subsequent tableaux 
ranks = [0,1] #We know the ranks of 0D and D; will add to this list as we build tableaux (but same as widths-1)
heights = [0,g-k+1] #Record the size of lambda^c 



def tabc(i): #Build the tableau corresponding to iD #Must be run for all i>1, in order, since it's recursive
    #Try to make it j rows shorter and wider than lambda(i-1) for j starting at 1 and less than k
    j = 1 #=k-rank jump
    
    if tryj(i,j) == Tableau([]):
        return Tableau([])
    
    while tryj(i,j).is_rectangular() == False:  
        if j < k-1:
            j+=1
            
        else: 
            return Tableau([])       
    return tryj(i,j)
        
#Calculate rank sequence 
for c in range(2,(2*g-2)):
    thistableau = tabc(c)
    if thistableau == Tableau([]):
        break
    else:
        heights.append(thistableau.height())
        ranks.append(ranks[c-1]+k-(heights[c-1]-heights[c]))
        print c, "D" 
        thistableau.pp()
    
                    
#Calculate Scrollar Invariants from rank sequence
sis = {i : [] for i in range(0,k)}
m = 0
i = 0
while i < k:
    if sis[i] == []:
        if all(ranks[c] >= i*c + i + c - m for c in range(0,len(ranks))):
            sis[i] = m
            i += 1
        else: 
            m += 1


#Start with aisum=1 so we don't have to subtract 1 in check every time
#aisum = 1
#i = 1
#t = 1
#while i < k+1:
#    if sis[i] == []:
#        if all(ranks[v] >= i*t + i -t -aisum for v in range(t,len(ranks))):
#            sis[i] = t-1
#            aisum += t-1
#            i += 1
#        else:
#            t += 1
        
print "The heights of cD are:", heights
print "The rank sequence is:", ranks
print "The scrollar invariants are:",sis


KeyboardInterrupt: 